In [ ]:
pip install git+https://github.com/pishiko/pytube.git@42a7d8322dd7749a9e950baf6860d115bbeaedfc

In [ ]:
pip install ffmpeg

In [ ]:
from pytube import YouTube
from pytube import Channel
import os
import re
import datetime
import csv
import pandas as pd
import time

In [ ]:
from urllib.error import HTTPError
def download(link, path, last_run_date):

  within_timeframe = True

  video = YouTube(link)
  try:
      print(video.check_availability())
      print(video.title())
      if video.length > 300:
        print("video too long (probably not a beat)")
        return within_timeframe

      if video.publish_date < last_run_date:
        print("no new beats")
        within_timeframe = False
        return within_timeframe

      bpm_search = re.search(r'(?i)\b(?:BPM|bpm)[:\s]?\s*(\d{2,3})\b|\b(\d{2,3})\s?BPM\b', video.description)
      if bpm_search:
        bpm = bpm_search.group(1) or bpm_search.group(2)

        audio_track = video.streams.filter(only_audio=True).first()
        mp4_file = audio_track.download(output_path=path)
        file_name = audio_track.default_filename

        source = path + file_name
        os.rename(source, source.replace(' ', '_'))
        file_name = source.replace(' ','_')
        file_without_ext, ext = os.path.splitext(file_name)
        file_without_ext += "---"+bpm+"-bpm"

        command = f"ffmpeg -i {file_name} {file_without_ext}.mp3"

        os.system(command)
        os.remove(file_name)


      else:
        print('could not find bpm')
      return within_timeframe

  except TypeError as te:
    print('type error', te)
    return False
  except HTTPError as err:
    if err.code == 429:
      print('rate limited :/ pls abort')
      return False
    else:
      print('http error', err)
      return False

In [ ]:
path = '/content/'
link_base = 'https://www.youtube.com/channel/'

df = pd.read_csv('/content/ytdownloader_channels.csv', header=None)
df.dropna(inplace=True)
csv_date = datetime.datetime.strptime(df[1][0], '(%Y, %m, %d, %H, %M, %S, %f)')
last_run_date = max(csv_date, datetime.datetime.now() - datetime.timedelta(days=7))

count = 0
for i in range(1,len(df)):
  print("producer {}/{}, {}".format(i, len(df)-1, df[0][i]))
  channel_link = link_base + df[1][i]
  c = Channel(channel_link)
  for url in c.video_urls:
    valid = download(url,path,last_run_date)
    if valid:
      count += 1
    else:
      break

    print(url)

df[1][0] = datetime.datetime.now().strftime('(%Y, %m, %d, %H, %M, %S, %f)')
df.to_csv(path+'/ytdownloader_channels.csv', index=False, header=False)
print('{} beats downloaded'.format(count))

# notes for reference

In [ ]:
substring = "150BPM, bpm: 78, BPM: 148, BPM:105, Bpm: 130, BPM:  146, 190 BPM, 160 bpm, 143bpm"
matches = re.finditer(r'(?i)\b(?:BPM|bpm)[:\s]?\s*(\d{2,3})\b|\b(\d{2,3})\s?BPM\b', substring)
for match in matches:
    print(match.group(1) or match.group(2))

To get channel IDs:

https://yt.lemnoslife.com/channels?handle=farberbeats

could also read in one video and then do video.channel_id for the rest